In [3]:
import pandas as pd
import requests
from datetime import datetime
import yaml
import json
import ipywidgets as widgets
import time
import pickle
import os
import pyperclip

# 가변 정보(yaml)
변경 시 yaml 파일 내에서 변경

In [16]:
# load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
client_id = config['data_crawling']['client_info']['client_id']
client_secret = config['data_crawling']['client_info']['client_secret']
exclude_phrase = config['data_crawling']['exclude_phrase'] # list
exclude_hashtag = config['data_crawling']['exclude_hashtag'] # list

# 불변 정보
변경 시 python 코드 내에서 변경

In [17]:
# 네이버 api 기본 주소
api_address = "https://openapi.naver.com/v1/search/"

# 크롤링 대상
targets = ["blog", "cafearticle"]
queries = ["스탠바이미", "스탠바이미고"]

# 파일 형태
format = "json"

# 검색 결과 개수(default: 10, max: 100)
display = 100

# 검색 결과 정렬(sim: 정확도순으로 내림차순 정렬(기본값) date: 날짜순으로 내림차순 정렬)
sort = "sim"

In [24]:
# url 생성
url = api_address + targets[0] + "." + format

# 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 파라미터 설정
params = {
    "query": queries[0],
    "display": display,
    "sort": sort,
    "start": 1
}

In [25]:
# api 호출
response = requests.get(url, headers=headers, params=params)

In [26]:
response.json()

{'lastBuildDate': 'Mon, 05 Aug 2024 20:25:09 +0900',
 'total': 59411,
 'start': 1,
 'display': 100,
 'items': [{'title': '리오다 이동식 티비스탠드 <b>스탠바이미</b> 무드 실현',
   'link': 'https://blog.naver.com/cheawhi/223534142774',
   'description': 'LG TV와 이동식 티비스탠드로 실현한 <b>스탠바이미</b> st 리오다 마리 티비거치대 티비를 구매했을 때만해도 나는 거실에 TV를 두지 않고 큰 테이블을 놓아 카페처럼 활용하겠다는 마음으로 게임용 모델을... ',
   'bloggername': '호담의 맥시멀라이프',
   'bloggerlink': 'blog.naver.com/cheawhi',
   'postdate': '20240802'},
  {'title': '2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨... ',
   'link': 'https://www.khaiyang.com/4636',
   'description': '2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨, 돌비 비전 애트모스) Khai IT리뷰 / 디지털.가전 2024. 5. 8. 20:53 사고 싶어도 사지 못하는 핫 아이템인 이동식 라이프 스타일 스크린 LG <b>스탠바이미</b>... ',
   'bloggername': '바람의 이야기, 카이',
   'bloggerlink': 'https://www.khaiyang.com/',
   'postdate': '20240508'},
  {'title': '콩비지와 옥수수로 만든 고양이모래 : <b>스탠바이미</b> 모래영웅... ',
   'link': 'https://blog.naver.com/birthday7725/223501340397',
   'description': '▲ <b>스탠바

In [28]:
response.json().get('items')

[{'title': '리오다 이동식 티비스탠드 <b>스탠바이미</b> 무드 실현',
  'link': 'https://blog.naver.com/cheawhi/223534142774',
  'description': 'LG TV와 이동식 티비스탠드로 실현한 <b>스탠바이미</b> st 리오다 마리 티비거치대 티비를 구매했을 때만해도 나는 거실에 TV를 두지 않고 큰 테이블을 놓아 카페처럼 활용하겠다는 마음으로 게임용 모델을... ',
  'bloggername': '호담의 맥시멀라이프',
  'bloggerlink': 'blog.naver.com/cheawhi',
  'postdate': '20240802'},
 {'title': '2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨... ',
  'link': 'https://www.khaiyang.com/4636',
  'description': '2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨, 돌비 비전 애트모스) Khai IT리뷰 / 디지털.가전 2024. 5. 8. 20:53 사고 싶어도 사지 못하는 핫 아이템인 이동식 라이프 스타일 스크린 LG <b>스탠바이미</b>... ',
  'bloggername': '바람의 이야기, 카이',
  'bloggerlink': 'https://www.khaiyang.com/',
  'postdate': '20240508'},
 {'title': '콩비지와 옥수수로 만든 고양이모래 : <b>스탠바이미</b> 모래영웅... ',
  'link': 'https://blog.naver.com/birthday7725/223501340397',
  'description': '▲ <b>스탠바이미</b>펫에서 구매 할 수 있고요, 캣플러스/매일멍냥/쿠팡 등에서도 구매 가능해요. 판매처가... 귀여움이 군데군데서 발견되는 고양이모래, <b>스탠바이미</b> 모래영웅 두부모래. 엄마 고양이랑 아기... ',
  '

In [18]:
# 결과 처리
if response.status_code == 200:  # HTTP 응답 코드가 200이면 성공
    result = response.json()  # JSON 형태로 응답을 파싱
    print("Success")
else:
    print("Error Code:", response.status_code)  # 에러 코드 출력

Success


# 수집

In [1]:
import pandas as pd
import requests
from datetime import datetime
import yaml
import json
import ipywidgets as widgets
import time
import pickle
import os

In [38]:
# load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
client_id = config['data_crawling']['client_info']['client_id']
client_secret = config['data_crawling']['client_info']['client_secret']
exclude_phrase = config['data_crawling']['exclude_phrase'] # list
exclude_hashtag = config['data_crawling']['exclude_hashtag'] # list

In [3]:
# 네이버 api 기본 주소
api_address = "https://openapi.naver.com/v1/search/"

# 크롤링 대상
targets = ["blog", "cafearticle"]
queries = ["스탠바이미", "스탠바이미고"]

# 파일 형태
format = "json"

# 검색 결과 개수(default: 10, max: 100)
display = 100

# 검색 결과 정렬(sim: 정확도순으로 내림차순 정렬(기본값) date: 날짜순으로 내림차순 정렬)
sort = "sim"

In [4]:
# 네이버 API에서 데이터를 가져오는 함수
def fetch_data(target, query, start, display, sort):
    url = api_address + target + "." + format
    params = {
        "query": query,
        "display": display,
        "sort": sort,
        "start": start
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error Code: {response.status_code}")
        return None

In [5]:
# def clean_html_tags(text):
#     if text:
#         # '<b>' 및 '</b>' 태그를 제거
#         text = re.sub(r'<\/?b>', '', text)
#     return text

In [29]:
# 데이터 수집
all_data = []  # 수집한 모든 데이터를 저장할 리스트

for target in targets:
    for query in queries:
        start = 1  # 첫 번째 항목부터 시작
        while start <= 1000:
            print(f"Fetching data for target: {target}, query: {query}, start: {start}")
            result = fetch_data(target, query, start, display, sort)
            if result:
                items = result.get('items', [])
                for item in items:
                    item['target'] = target
                    item['query'] = query
                all_data.extend(items)
                start += display  # 다음 결과 셋으로 이동
                time.sleep(0.1)  # 요청 간 지연 추가
            else:
                break
            # 필요한 만큼의 항목이 수집되면 종료



Fetching data for target: blog, query: 스탠바이미, start: 1
Fetching data for target: blog, query: 스탠바이미, start: 101
Fetching data for target: blog, query: 스탠바이미, start: 201
Fetching data for target: blog, query: 스탠바이미, start: 301
Fetching data for target: blog, query: 스탠바이미, start: 401
Fetching data for target: blog, query: 스탠바이미, start: 501
Fetching data for target: blog, query: 스탠바이미, start: 601
Fetching data for target: blog, query: 스탠바이미, start: 701
Fetching data for target: blog, query: 스탠바이미, start: 801
Fetching data for target: blog, query: 스탠바이미, start: 901
Fetching data for target: blog, query: 스탠바이미고, start: 1
Fetching data for target: blog, query: 스탠바이미고, start: 101
Fetching data for target: blog, query: 스탠바이미고, start: 201
Fetching data for target: blog, query: 스탠바이미고, start: 301
Fetching data for target: blog, query: 스탠바이미고, start: 401
Fetching data for target: blog, query: 스탠바이미고, start: 501
Fetching data for target: blog, query: 스탠바이미고, start: 601
Fetching data for target: bl

In [30]:
# 데이터 저장
df = pd.DataFrame(all_data)
df

,title,link,description,bloggername,bloggerlink,postdate,target,query,cafename,cafeurl
0,리오다 이동식 티비스탠드 <b>스탠바이미</b> 무드 실현,https://blog.naver.com/cheawhi/223534142774,LG TV와 이동식 티비스탠드로 실현한 <b>스탠바이미</b> st 리오다 마리 티...,호담의 맥시멀라이프,blog.naver.com/cheawhi,20240802,blog,스탠바이미,NaN,NaN
1,"2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨...",https://www.khaiyang.com/4636,"2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨,...","바람의 이야기, 카이",https://www.khaiyang.com/,20240508,blog,스탠바이미,NaN,NaN
2,콩비지와 옥수수로 만든 고양이모래 : <b>스탠바이미</b> 모래영웅...,https://blog.naver.com/birthday7725/223501340397,"▲ <b>스탠바이미</b>펫에서 구매 할 수 있고요, 캣플러스/매일멍냥/쿠팡 등에서...",나의 하루와 고양이의 하루,blog.naver.com/birthday7725,20240704,blog,스탠바이미,NaN,NaN
3,"GJ의 Good Life 18 - 서령, 바베큐, 캠풍(ft.<b>스탠바이미</b> 고)",https://blog.naver.com/good2g0603/223482257555,캠풍 <b>스탠바이미</b> 고 난 캠핑을 선호하지 않는다. 뭔가 가득 챙겨가는 것...,Good 2 Great,blog.naver.com/good2g0603,20240617,blog,스탠바이미,NaN,NaN
4,LG 포터블 스크린 2세대 <b>스탠바이미</b> GO 27ART10CKPL 렌탈...,https://blog.naver.com/creamysorrow/223500577360,시중에 판매되는 여러 브랜드를 검색해 비교하고 가격과 제품력을 비교해 본 결과 LG...,"밝음, 빛 그리고 에너지",blog.naver.com/creamysorrow,20240704,blog,스탠바이미,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3995,스탠비이미고 새상품 판매,http://cafe.naver.com/chocammall/9963627,초캠장터 이용규칙 및 상거래 이용약관 https://cafe.naver.com/ch...,NaN,NaN,NaN,cafearticle,스탠바이미고,초캠장터(캠핑퍼스트장터),https://cafe.naver.com/chocammall
3996,삼성스토어 강서점에서 가전 졸업했어요&gt;&lt;,http://cafe.naver.com/b00k2012/1048315,"저는 통돌이세탁기를 선호하고, 저도 예랑이도 TV를 보지 않아서 <b>스탠바이미</...",NaN,NaN,NaN,cafearticle,스탠바이미고,다이렉트웨딩카페-제이웨딩 결혼준비싸...,https://cafe.naver.com/b00k2012
3997,욕심많은 롱프로 2인 igt배치 답이없나..?,http://cafe.naver.com/snowpeaklove/166013,"그 외 <b>스탠바이미고</b>, 물건담는 박스(75L), 쿨러, 기어미션 난로 등...",NaN,NaN,NaN,cafearticle,스탠바이미고,스노우피크 유저들의 모임,https://cafe.naver.com/snowpeaklove
3998,<b>스탠바이미고</b> 받침대!!,http://cafe.naver.com/campingkan/541872,"다들 <b>스탠바이미고</b>는 어디에 올려놓고 쓰시는 지 궁금했는데, 막 비싼 테...",NaN,NaN,NaN,cafearticle,스탠바이미고,캠핑칸 campingkan,https://cafe.naver.com/campingkan


In [36]:
df.columns

Index(['title', 'link', 'description', 'bloggername', 'bloggerlink',
       'postdate', 'target', 'query', 'cafename', 'cafeurl'],
      dtype='object')

In [37]:
df.to_csv('naver_blog_crawling_4000.csv', index=False, encoding='utf-8-sig')

# 전처리

### 카페

In [39]:
# df cafeurl 중 lgstanbymeclub, imsanbu, campingfirst 있는 행만 추출
df_cafe_filter = df[df['link'].str.contains('lgstanbymeclub|imsanbu|campingfirst')]

In [2]:
df_cafe_filter.reset_index(drop=True, inplace=True)
df_cafe_filter

NameError: name 'df_cafe_filter' is not defined

In [42]:
df_cafe_filter['cafename'].value_counts()

cafename
캠핑퍼스트(초보캠핑)                  172
맘스홀릭 베이비 (330만 엄마들의 임신...     12
LG 스탠바이미 공식카페                  8
Name: count, dtype: int64

### 블로그

In [43]:
exclude_phrase

['제품을 협찬받아 작성되었습니다',
 '제품을 제공받아 작성하였습니다',
 '협찬을 받아 작성된 포스팅입니다',
 '소정의 사례금을 받고 작성되었습니다',
 '사례금을 받아 작성한 글입니다',
 '제공한 금전적 보상을 받고 작성되었습니다',
 '제품을 협찬받고 소정의 사례금을 받아 작성되었습니다',
 '협찬과 사례금을 받아 작성된 리뷰입니다',
 '제품 협찬과 함께 작성되었습니다',
 '제품과 사례금을 제공받아 작성되었습니다',
 '광고를 포함하고 있습니다',
 '홍보를 목적으로 작성되었습니다']

In [44]:
exclude_hashtag

['#광고',
 '#협찬',
 '#스폰서',
 '#지원받음',
 '#유료광고',
 '#광고포함',
 '#제휴',
 '#파트너십',
 '#홍보',
 '#광고제휴',
 '#광고리뷰',
 '#프로모션',
 '#AD',
 '#Sponsored',
 '#SponsoredPost',
 '#PaidPartnership',
 '#PaidPromotion',
 '#Promotion',
 '#PR']

In [45]:
def contains_exclude_terms(description, phrases, hashtags):
    for phrase in phrases:
        if phrase in description:
            return True
    for hashtag in hashtags:
        if hashtag in description:
            return True
    return False

In [46]:
# Filtering the DataFrame
df_exclude = df[~df['description'].apply(lambda desc: contains_exclude_terms(desc, exclude_phrase, exclude_hashtag))]

In [48]:
df_exclude.reset_index(drop=True, inplace=True)

In [49]:
df_exclude

,title,link,description,bloggername,bloggerlink,postdate,target,query,cafename,cafeurl
0,리오다 이동식 티비스탠드 <b>스탠바이미</b> 무드 실현,https://blog.naver.com/cheawhi/223534142774,LG TV와 이동식 티비스탠드로 실현한 <b>스탠바이미</b> st 리오다 마리 티...,호담의 맥시멀라이프,blog.naver.com/cheawhi,20240802,blog,스탠바이미,NaN,NaN
1,"2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨...",https://www.khaiyang.com/4636,"2024년 신형 LG <b>스탠바이미</b> 뭐가 달라졌나? (가격, 미니 리모컨,...","바람의 이야기, 카이",https://www.khaiyang.com/,20240508,blog,스탠바이미,NaN,NaN
2,콩비지와 옥수수로 만든 고양이모래 : <b>스탠바이미</b> 모래영웅...,https://blog.naver.com/birthday7725/223501340397,"▲ <b>스탠바이미</b>펫에서 구매 할 수 있고요, 캣플러스/매일멍냥/쿠팡 등에서...",나의 하루와 고양이의 하루,blog.naver.com/birthday7725,20240704,blog,스탠바이미,NaN,NaN
3,"GJ의 Good Life 18 - 서령, 바베큐, 캠풍(ft.<b>스탠바이미</b> 고)",https://blog.naver.com/good2g0603/223482257555,캠풍 <b>스탠바이미</b> 고 난 캠핑을 선호하지 않는다. 뭔가 가득 챙겨가는 것...,Good 2 Great,blog.naver.com/good2g0603,20240617,blog,스탠바이미,NaN,NaN
4,LG 포터블 스크린 2세대 <b>스탠바이미</b> GO 27ART10CKPL 렌탈...,https://blog.naver.com/creamysorrow/223500577360,시중에 판매되는 여러 브랜드를 검색해 비교하고 가격과 제품력을 비교해 본 결과 LG...,"밝음, 빛 그리고 에너지",blog.naver.com/creamysorrow,20240704,blog,스탠바이미,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3993,스탠비이미고 새상품 판매,http://cafe.naver.com/chocammall/9963627,초캠장터 이용규칙 및 상거래 이용약관 https://cafe.naver.com/ch...,NaN,NaN,NaN,cafearticle,스탠바이미고,초캠장터(캠핑퍼스트장터),https://cafe.naver.com/chocammall
3994,삼성스토어 강서점에서 가전 졸업했어요&gt;&lt;,http://cafe.naver.com/b00k2012/1048315,"저는 통돌이세탁기를 선호하고, 저도 예랑이도 TV를 보지 않아서 <b>스탠바이미</...",NaN,NaN,NaN,cafearticle,스탠바이미고,다이렉트웨딩카페-제이웨딩 결혼준비싸...,https://cafe.naver.com/b00k2012
3995,욕심많은 롱프로 2인 igt배치 답이없나..?,http://cafe.naver.com/snowpeaklove/166013,"그 외 <b>스탠바이미고</b>, 물건담는 박스(75L), 쿨러, 기어미션 난로 등...",NaN,NaN,NaN,cafearticle,스탠바이미고,스노우피크 유저들의 모임,https://cafe.naver.com/snowpeaklove
3996,<b>스탠바이미고</b> 받침대!!,http://cafe.naver.com/campingkan/541872,"다들 <b>스탠바이미고</b>는 어디에 올려놓고 쓰시는 지 궁금했는데, 막 비싼 테...",NaN,NaN,NaN,cafearticle,스탠바이미고,캠핑칸 campingkan,https://cafe.naver.com/campingkan


In [53]:
tmp = df[df['description'].apply(lambda desc: contains_exclude_terms(desc, exclude_phrase, exclude_hashtag))].reset_index(drop=True)

In [1]:
tmp

NameError: name 'tmp' is not defined

In [4]:
tmp['description'][0]

NameError: name 'tmp' is not defined

In [56]:
tmp['description'][1]

'생긴 <b>스탠바이미 고</b>는 ‘캠핑족’을 겨냥한 제품이다. 가방 한쪽 면에 대형 스크린과 스탠드가 붙어... 엘지는 인터넷에서만 난리임 #광고기사 #광고 #스탠바이미 #<b>스탠바이미고</b> #LG전자 #캠핑족'